In [ ]:
import cv2
import imutils #it provides an easy way for functions like resizing,rotation,translation
import numpy as np
from collections import deque #to keep a track of the trails of the object
import time

In [ ]:
#the HSV color range for skin color(which works for me, it might be different for you)
skin_lower = (0, 10, 60) 
skin_upper = (20, 150, 255)

#Used in deque structure to store no. of given buffer points
buffer = 30 #mainly to store number of buffer points

#Points deque structure storing 'buffer' no. of object coordinates
pts = deque(maxlen = buffer)
#Counts the minimum no. of frames to be detected where direction change occurs
counter = 0
#Change in direction is stored in dX, dY
(dX, dY) = (0, 0)
#Variable to store direction string
direction = ''
flag = 0

#Start video capture
video_capture = cv2.VideoCapture(0) 

#Sleep for 2 seconds to let camera initialize properly.
time.sleep(2)

width,height = pyautogui.size()

#Loop until OpenCV window is not closed
while True:
    ret,frame = video_capture.read()#reading the frames
    frame = cv2.flip(frame,1) #flipping to avoid the mirror effect
    #resize to smaller dimension to get faster frames per seconds(you can try any to get the best speed)
    frame = imutils.resize(frame, width = 600)
    blurred_frame = cv2.GaussianBlur(frame, (5,5), 0) #to reduce the external noise and focus on the structure of the object
    #Convert the frame to HSV since hsv helps in seperating the color information unlike RGB which is mainly red,blue,green
    hsv_converted_frame = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_converted_frame, skin_lower, skin_upper) #segment the region under the given lower and upper range
    #you can further erode or dilate to remove some finer details like white dots

    #find all contours in the masked image
    #you can pass cv2.CHAIN_APPROX_NONE to detect all the boundary points which will take more memory
    _,cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    center = None #initialize centre to be zero 

    if(len(cnts) > 0): #checks for minimum one contour
        #Find the contour with maximum area
        c = max(cnts, key = cv2.contourArea)
        #since im focussing on my palm, calculate the contour for a cicle
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        #Calculate the centroid 
        M = cv2.moments(c)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        if radius > 10:
            #Draw circles around the object as well as its centre
            cv2.circle(frame, (int(x), int(y)), int(radius), (0,255,255), 2) #for the outer circle
            cv2.circle(frame, center, 5, (0,255,255), -1) #for the centroid
            #Append the detected object in the frame to pts deque structure
            pts.appendleft(center) #append the centroid to the deque

    for i in np.arange(1, len(pts)):
        #If no points are detected, move on.
        if(pts[i-1] == None or pts[i] == None):
            continue

        #If atleast 10 frames have direction change, proceed
        if counter >= 10 and i == 1 and pts[-10] is not None:
            #Calculate the distance between the current frame and 10th frame before
            dX = pts[-10][0] - pts[i][0]
            dY = pts[-10][1] - pts[i][1]
            (dirX, dirY) = ('', '')

            #If distance is greater than 100 pixels, considerable direction change has occured.
            if np.abs(dX) > 100:
                dirX = 'Left' if np.sign(dX) == 1 else 'Right'

            if np.abs(dY) > 50:
                dirY = 'Up' if np.sign(dY) == 1 else 'Down'
                
            if dirX != "" and dirY != "":
                direction = "{}-{}".format(dirY, dirX)

            #Set direction variable to the detected direction
            direction = dirX if dirX != '' else dirY

        #Draw a trailing red line to depict motion of the object.
        thickness = int(np.sqrt(buffer / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
        
    cv2.putText(frame, direction, (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

    cv2.imshow('Window Direction Detection', frame)
    key = cv2.waitKey(1) & 0xFF
    #Update counter as the direction change has been detected.
    counter += 1
        
    #If q is pressed, close the window
    if(key == ord('q')):
        break
        
video_capture.release()
cv2.destroyAllWindows()
